In [1]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='15cfd96b0bdfd1b0c67597c24f91907776c02a6d', drake_version='0.27.0', drake_build='release')

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc_planar, zmq_url_planar, web_url_planar = start_zmq_server_as_subprocess(server_args=server_args)
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np
from ipywidgets import FloatSlider, ToggleButton
from IPython.display import display, SVG, HTML
import pydot

import pydrake.all
from pydrake.all import (
    MultibodyPlant, DiagramBuilder, LinearQuadraticRegulator, Saturation, SceneGraph, Simulator, 
    WrapToSystem, AddMultibodyPlantSceneGraph, Parser, MathematicalProgram, eq, PiecewisePolynomial,
    SnoptSolver, TrajectorySource, MultibodyPositionToGeometryPose, PlanarSceneGraphVisualizer
)
from pydrake.examples.acrobot import AcrobotPlant, AcrobotGeometry
from pydrake.systems.jupyter_widgets import WidgetSystem
from pydrake.common.containers import namedview
from underactuated import FindResource, ManipulatorDynamics
from underactuated.jupyter import running_as_notebook


Cloning into '/opt/underactuated'...

HEAD is now at 15cfd96 and again

ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
ERROR: bokeh 2.3.2 has requirement pillow>=7.1.0, but you'll have pillow 7.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.






## Problem Description

We want to write a nonlinear optimization for the ball balancing bot. In order to complete our trajectory optimization, we will use `MathematicalProgram`.

**maybe write more later if we submit the notebook

## Parse the `urdf` and Get the `MultibodyPlant`

We start by defining a couple of physical parameters that we will need below.

In [30]:
ballbot_standard = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="5" />
      <inertia ixx=".2" ixy="0" ixz="0" iyy="0.2" iyz="0" izz="0.2" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.10" ixy="0" ixz="0" iyy="0.18" iyz="0" izz="0.1" />
    </inertial>
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>
   

  <joint name="theta1" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>


  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta1" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.1</mechanicalReduction>
  </transmission>
</robot>
"""

#here the ball is 20% heavier and has 20% more moment of inertia
ballbot_heavyball = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="6" />
      <inertia ixx=".24" ixy="0" ixz="0" iyy="0.24" iyz="0" izz="0.24" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.10" ixy="0" ixz="0" iyy="0.18" iyz="0" izz="0.1" />
    </inertial>
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>
   

  <joint name="theta1" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>


  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta1" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.1</mechanicalReduction>
  </transmission>
</robot>
"""

#here the ball is 20% smaller in radius
ballbot_smallball = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="5" />
      <inertia ixx=".2" ixy="0" ixz="0" iyy="0.2" iyz="0" izz="0.2" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".08" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.10" ixy="0" ixz="0" iyy="0.18" iyz="0" izz="0.1" />
    </inertial>
    <visual>
      <origin xyz="0 0 .04" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>
   

  <joint name="theta1" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>


  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta1" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.08</mechanicalReduction>
  </transmission>
</robot>
"""

## The Trajectory Optimization Problem

We start by setting some parameters of our optimization problem.

Here are part of the constraints of the optimization problem:


1.   Time steps need to be bounded between h_min and h_max
2.   Map configurations, velocities, accelerations to each other using the Euler method
3.   Something about dynamics
4.   Want the thing on top to be horizonal at the end (theta1 = 0, theta2 = 0)
5.   Can add an angle bound (like it can't be more than 15 deg away from horizontal)



Here we set the initial guess for our optimization problem.

For the time steps `h` we just initialize them to their maximal value `h_max` (somewhat an arbitrary decision, but it works).

For the robot configuration `q`, we interpolate between the initial value `q0_guess` and the final value `- q0_guess`.
In our implementation, the value given below for `q0_guess` made the optimization converge.
But, if you find the need, feel free to tweak this parameter.
The initial guess for the velocity and the acceleration is obtained by differentiating the one for the position.

The normal force `f` at the stance foot is equal to the total `weight` of the robot.

All the other optimization variables are initialized at zero.
(Note that, if the initial guess for a variable is not specified, the default value is zero.)

We can finally solve the problem! Be sure that the solver actually converged: you can check this by looking at the variable result.is_success() (printed below).

In the following cell we retrieve the optimal value of the decision variables.

## Animate the Result

Here we quickly build a Drake diagram to animate the result we got from trajectory optimization: useful for debugging your code and to be sure that everything looks good.

In [31]:
class optimization_step:
  def __init__(self, ballbot_urdf, start, goal, T, actuation_lim, tilt_lim, h_min = 0.005, 
               h_max = 0.5):
    self.ballbot_urdf = ballbot_urdf
    self.start = start
    self.goal = goal
    self.T = T
    self.actuation_lim = actuation_lim
    self.tilt_lim = tilt_lim
    self.h_min = h_min
    self.h_max = h_max

  def create_plant(self):
    self.ballbot_2D = MultibodyPlant(time_step=0)
    Parser(self.ballbot_2D).AddModelFromString(self.ballbot_urdf, "urdf")
    self.ballbot_2D.Finalize()

    # overwrite MultibodyPlant with its autodiff copy
    self.ballbot_2D = self.ballbot_2D.ToAutoDiffXd() 

    # number of configuration variables
    self.nq = self.ballbot_2D.num_positions()  # Returns the size of the multibody system state vector x = [q v]. This will be num_positions() plus num_velocities()

  def ballbot2D_dynamics(self, vars):
    split_at = [self.nq, 2 * self.nq, 3 * self.nq]
    q, qd, qdd, u = np.split(vars, split_at)
    
    # set compass gait state
    context = self.ballbot_2D.CreateDefaultContext()
    self.ballbot_2D.SetPositions(context, q)
    self.ballbot_2D.SetVelocities(context, qd)
    self.ballbot_2D.get_actuation_input_port().FixValue(context, u)
    
    # matrices for the manipulator equations
    M, Cv, tauG, B, tauExt = ManipulatorDynamics(self.ballbot_2D,q,qd)
    theta = q[1]

    # return violation of the manipulator equations
    return M.dot(qdd) + Cv - tauG -B@u - tauExt

  def generate_cont_vars(self, prog):

    """ Returns h, q, qd, qdd, u """
    # vector of the time intervals
    # (distances between the T + 1 break points)
    T = self.T
    nq = self.nq
    h = prog.NewContinuousVariables(T, name='h')

    # system configuration, generalized velocities, and accelerations
    q = prog.NewContinuousVariables(rows=T+1, cols=nq, name='q')
    qd = prog.NewContinuousVariables(rows=T+1, cols=nq, name='qd')
    qdd = prog.NewContinuousVariables(rows=T, cols=nq, name='qdd')

    # actuation
    u = prog.NewContinuousVariables(rows=T, cols=nq, name='u')

    return h, q, qd, qdd, u

  def set_initial_guess(self, prog, h_max, T, h, q, qd, qdd):
    # vector of the initial guess
    initial_guess = np.empty(prog.num_vars())

    # initial guess for the time step
    h_guess = h_max
    prog.SetDecisionVariableValueInVector(h, [h_guess] * T, initial_guess)

    # linear interpolation of the configuration
    q0_guess = np.array([0, .15])
    q_guess_poly = PiecewisePolynomial.FirstOrderHold(
        [0, T * h_guess],
        np.column_stack((q0_guess, - q0_guess))
    )
    qd_guess_poly = q_guess_poly.derivative()
    qdd_guess_poly = q_guess_poly.derivative()

    # set initial guess for configuration, velocity, and acceleration
    q_guess = np.hstack([q_guess_poly.value(t * h_guess) for t in range(T + 1)]).T
    qd_guess = np.hstack([qd_guess_poly.value(t * h_guess) for t in range(T + 1)]).T
    qdd_guess = np.hstack([qdd_guess_poly.value(t * h_guess) for t in range(T)]).T
    prog.SetDecisionVariableValueInVector(q, q_guess, initial_guess)
    prog.SetDecisionVariableValueInVector(qd, qd_guess, initial_guess)
    prog.SetDecisionVariableValueInVector(qdd, qdd_guess, initial_guess)

    return initial_guess

  def get_states(self, solution):
    T = self.T
    result, h, q, qd, qdd, u, success = solution
    h_opt = result.GetSolution(h)
    q_opt = result.GetSolution(q)
    qd_opt = result.GetSolution(qd)
    qdd_opt = result.GetSolution(qdd)
    u_opt = result.GetSolution(u)

    # stack states
    x_opt = np.hstack((q_opt, qd_opt))

    # interpolate u values for animation
    time_breaks_opt = np.array([sum(h_opt[:t]) for t in range(T+1)])
    x_opt_poly = PiecewisePolynomial.FirstOrderHold(time_breaks_opt, x_opt.T)
    return x_opt_poly


  def simulate(self, result, h, q, qd, qdd, u, success):
    T = self.T
    # get optimal solution
    h_opt = result.GetSolution(h)
    q_opt = result.GetSolution(q)
    qd_opt = result.GetSolution(qd)
    qdd_opt = result.GetSolution(qdd)

    # stack states
    x_opt = np.hstack((q_opt, qd_opt))

    # interpolate state values for animation
    time_breaks_opt = np.array([sum(h_opt[:t]) for t in range(T+1)])
    x_opt_poly = PiecewisePolynomial.FirstOrderHold(time_breaks_opt, x_opt.T)

    print(f'shape of time_breaks_opt: {time_breaks_opt.shape}')
    print(time_breaks_opt)
    print(f'shape of x_opt: {x_opt.shape}')
    print(x_opt)

    # parse urdf with scene graph
    ballbot = MultibodyPlant(time_step=0)
    scene_graph = SceneGraph()
    ballbot.RegisterAsSourceForSceneGraph(scene_graph)
    Parser(ballbot).AddModelFromString(ballbot_urdf, "urdf")
    ballbot.Finalize()

    # build block diagram and drive system state with
    # the trajectory from the optimization problem
    builder = DiagramBuilder()
    source = builder.AddSystem(TrajectorySource(x_opt_poly))
    builder.AddSystem(scene_graph)
    pos_to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(ballbot, input_multibody_state=True))
    builder.Connect(source.get_output_port(0), pos_to_pose.get_input_port())
    builder.Connect(pos_to_pose.get_output_port(), scene_graph.get_source_pose_port(ballbot.get_source_id()))

    # add visualizer
    xlim = [-2, 2]
    ylim = [-.4, 1.5]
    visualizer = builder.AddSystem(PlanarSceneGraphVisualizer(scene_graph, xlim=xlim, ylim=ylim, show=False))
    builder.Connect(scene_graph.get_pose_bundle_output_port(), visualizer.get_input_port(0))
    simulator = Simulator(builder.Build())

    # generate and display animation
    visualizer.start_recording()
    simulator.AdvanceTo(x_opt_poly.end_time())
    ani = visualizer.get_recording_as_animation()
    return ani
  

    # --------------- Various Optimization Setups -----------

  def lqr_cost(self):
    T = self.T
    h_min = self.h_min
    h_max = self.h_max
    goal = self.goal
    start = self.start
    actuation_lim = self.actuation_lim
    tilt_lim = self.tilt_lim
    nq = self.nq

    # initialize program
    prog = MathematicalProgram()
    h, q, qd, qdd, u = self.generate_cont_vars(prog)

    # 1. lower and upper bound on the time steps for all t
    prog.AddBoundingBoxConstraint([h_min] * T, [h_max] * T, h)

    # 2. link the configurations, velocities, and accelerations
    # uses implicit Euler method, https://en.wikipedia.org/wiki/Backward_Euler_method
    for t in range(T):
        prog.AddConstraint(eq(q[t+1], q[t] + h[t] * qd[t+1]))
        prog.AddConstraint(eq(qd[t+1], qd[t] + h[t] * qdd[t]))

    # 3. drake dynamics
    for t in range(T):
      vars = np.concatenate((q[t], qd[t], qdd[t], u[t]))
      prog.AddConstraint(self.ballbot2D_dynamics, lb=[0]*nq, ub=[0]*nq, vars=vars)
      prog.AddLinearConstraint(u[t][0] == u[t][1])

    # 4. Penalize time/velocity away from goal
    total = 0
    for t in range(T):
      total += 10*(q[t][0] - goal[0])**2 # Penalize distance from goal
      total += 10*(q[t][1] - goal[1])**2
      total +=   (qd[t][0] - goal[2])**2 # Penalize velocity
      total +=   (qd[t][1] - goal[3])**2
      total +=     u[t][0]**2            # Penalize actuation
      prog.AddCost(total)  

    # 7. enforce starting position
    prog.AddLinearConstraint(q[0][0] == start[0])
    prog.AddLinearConstraint(q[0][1] == start[1])
    prog.AddLinearConstraint(qd[0][0] == start[2])
    prog.AddLinearConstraint(qd[0][1] == start[3])

    # 8. enforce goal position
    #prog.AddLinearConstraint(q[-1][0] == goal[0])
    #prog.AddLinearConstraint(q[-1][1] == goal[1])
    #prog.AddLinearConstraint(qd[-1][0] == goal[2])
    #prog.AddLinearConstraint(qd[-1][1] == goal[3])

    # 9. actuation constraint
    if actuation_lim is not None:
      for t in range(T):
        vars = np.concatenate((q[t], qd[t], qdd[t], u[t]))
        prog.AddLinearConstraint(u[t][0] <= actuation_lim)
        prog.AddLinearConstraint(u[t][0] >= -1* actuation_lim)

    # 10. Tilt Angle Constraint
    if tilt_lim is not None:
      for t in range(T):
        prog.AddLinearConstraint(q[t][1] <= tilt_lim)
        prog.AddLinearConstraint(q[t][1] >= -1* tilt_lim)

    initial_guess = self.set_initial_guess(prog, h_max, T, h, q, qd, qdd)

    # solve mathematical program with initial guess
    solver = SnoptSolver()
    result = solver.Solve(prog, initial_guess)

    print(f'Solution found? {result.is_success()}.')

    return result, h, q, qd, qdd, u, result.is_success()

In [32]:
start = [1, 0, 0, 0]
goal = [0, 0, 0, 0]
T = 50
actuation_lim = 100
tilt_lim = None
ballbot_urdf = ballbot_standard

step = optimization_step(ballbot_urdf, start, goal, T, actuation_lim, tilt_lim)
step.create_plant()
solution = step.lqr_cost()
states = step.get_states(solution)

HTML(step.simulate(*solution).to_jshtml())

Solution found? True.
shape of time_breaks_opt: (51,)
[0.         0.5        1.         1.5        2.         2.5
 3.         3.5        4.         4.5        5.         5.5
 6.         6.09949665 6.10449665 6.10949665 6.11449665 6.11949665
 6.12449665 6.12949665 6.13449665 6.13949665 6.14449665 6.14949665
 6.15449665 6.15949665 6.16449665 6.16949665 6.17449665 6.17949665
 6.18449665 6.18949665 6.19449665 6.19949665 6.20449665 6.20949665
 6.21449665 6.21949665 6.22449665 6.22949665 6.23449665 6.23949665
 6.24449665 6.24949665 6.25449665 6.25949665 6.26449665 6.26949665
 6.27449665 6.27949665 6.30100668]
shape of x_opt: (51, 4)
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 9.99492038e-01  4.47006580e-01 -1.01592404e-03  8.94013159e-01]
 [ 9.79012453e-01  5.03829892e-01 -4.09591690e-02  1.13646625e-01]
 [ 9.31928984e-01  4.16081862e-01 -9.41669394e-02 -1.75496060e-01]
 [ 8.62183357e-01  3.06714883e-01 -1.39491253e-01 -2.18733958e-01]
 [ 7.75482807e-01  2.07685927e

In [35]:
def model_predictive_control(ballbot_urdf, start, goal, optimizer, time_steps, horizon, actuation_lim = None, tilt_lim = None):
  # 1. take current state (or initial condition)
  # 2. solve optimization problem
  # 3. take first commanded u from problem
  # 3. add actuator noise
  # 4. execute for a step
  # 5. add state noise
  # self, ballbot_urdf, start, goal, T, actuation_lim, tilt_lim,
  
  nq = 4

  states = [start]
  time_intervals = []

  for i in range(time_steps):
    did_it_work = False
    result = None
    h = None
    q = None 
    qd = None
    qdd = None
    u = None

    while(not did_it_work):
      print(f'Beginning step {i} of {time_steps}...')
      # run trajectory optimizer with given initial conditions
      step = optimization_step(ballbot_urdf, states[-1], goal, horizon, actuation_lim, tilt_lim)
      step.create_plant()
      solution = optimizer(step)
      result, h, q, qd, qdd, u, did_it_work = solution

    # since the trajectory optimizer only works with discrete states in time,
    # it has no concept of time, and so uses the h[] decision variables to figure
    # how much physical time each timestep corresponds to. this is for numerical
    # convergence (i think) but it means we have the option of running the controller
    # at uniform points with variable spacing, or at uniform times. here, we choose states.
    #x_opt_poly = step.get_states(solution)
    #next_state = x_opt_poly.value(1).squeeze()
    q_opt = result.GetSolution(q)
    qd_opt = result.GetSolution(qd)
    h_opt = result.GetSolution(h)
    next_state = [q_opt[1][0], q_opt[1][1], qd_opt[1][0], qd_opt[1][1]]

    time_intervals.append(h_opt[0]) #don't know if this should be 0 or 1 tbh
    
  
    # print what's happenin so fischer can know where to direct his range
    print(f'step size: {time_intervals[-1]}')
    print(f'current state is: {states[-1]}')
    print(f'next state is: {next_state}\n')

    states.append(next_state)

  return np.vstack(states), time_intervals

In [36]:
start = [1, 0, 0, 0]
goal = [0, 0, 0, 0]
optimizer = optimization_step.lqr_cost
time_steps = 25
horizon = 40
state_noise = np.random.rand
actuator_noise = np.random.rand

mpc_result = model_predictive_control(ballbot_standard, start, goal, optimizer, time_steps, horizon)

Beginning step 0 of 25...
Solution found? True.
step size: 0.5
current state is: [1, 0, 0, 0]
next state is: [0.999504513119097, 0.436028455195033, -0.0009909737618068934, 0.872056910390066]

Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25...
Solution found? False.
Beginning step 1 of 25..

In [9]:
def mpc_simulator():
  states, time_intervals = mpc_result
  time_intervals = np.array(time_intervals)
  time_breaks_opt = np.array([sum(time_intervals[:t]) for t in range(26)])
  print(f'shape of time_breaks_opt: {time_breaks_opt.shape}')
  print(time_breaks_opt)
  print(f'shape of states: {states.shape}')
  print(states)
  x_opt_poly = PiecewisePolynomial.FirstOrderHold(time_breaks_opt, states.T)

  # parse urdf with scene graph
  ballbot = MultibodyPlant(time_step=0)
  scene_graph = SceneGraph()
  ballbot.RegisterAsSourceForSceneGraph(scene_graph)
  Parser(ballbot).AddModelFromString(ballbot_urdf, "urdf")
  ballbot.Finalize()

  # build block diagram and drive system state with
  # the trajectory from the optimization problem
  builder = DiagramBuilder()
  source = builder.AddSystem(TrajectorySource(x_opt_poly))
  builder.AddSystem(scene_graph)
  pos_to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(ballbot, input_multibody_state=True))
  builder.Connect(source.get_output_port(0), pos_to_pose.get_input_port())
  builder.Connect(pos_to_pose.get_output_port(), scene_graph.get_source_pose_port(ballbot.get_source_id()))

  # add visualizer
  xlim = [-2, 2]
  ylim = [-.4, 1.5]
  visualizer = builder.AddSystem(PlanarSceneGraphVisualizer(scene_graph, xlim=xlim, ylim=ylim, show=False))
  builder.Connect(scene_graph.get_pose_bundle_output_port(), visualizer.get_input_port(0))
  simulator = Simulator(builder.Build())

  # generate and display animation
  visualizer.start_recording()
  simulator.AdvanceTo(x_opt_poly.end_time())
  ani = visualizer.get_recording_as_animation()
  return ani

In [10]:
HTML(mpc_simulator().to_jshtml())

shape of time_breaks_opt: (26,)
[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5]
shape of states: (26, 4)
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 9.99504513e-01  4.36028455e-01 -9.90973762e-04  8.72056910e-01]
 [ 9.79431716e-01  4.84931338e-01 -4.01455939e-02  9.78057658e-02]
 [ 9.33646207e-01  3.89017913e-01 -9.15710175e-02 -1.91826850e-01]
 [ 8.66537524e-01  2.69391538e-01 -1.34217367e-01 -2.39252750e-01]
 [ 7.84392125e-01  1.58128056e-01 -1.64290798e-01 -2.22526964e-01]
 [ 6.93336687e-01  6.35970638e-02 -1.82110877e-01 -1.89061985e-01]
 [ 5.98711621e-01 -1.21089406e-02 -1.89250132e-01 -1.51412009e-01]
 [ 5.04846611e-01 -6.95027220e-02 -1.87730019e-01 -1.14787563e-01]
 [ 4.15036436e-01 -1.10404296e-01 -1.79620351e-01 -8.18031476e-02]
 [ 3.31626641e-01 -1.37553562e-01 -1.66819589e-01 -5.42985318e-02]
 [ 2.56164407e-01 -1.52777358e-01 -1.50924468e-01 -3.04475916e-02]
 [ 1.8